In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine

In [2]:
labels = pd.read_csv('data/round-1/mikey-devon-labels_validation-221126.csv')

In [3]:
labels

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2533,Devon Snyder,2533,479,University District,Occlusion,47.659710,-122.319855,bWNcSsunJ_MyQcFwru1edg,2022-11-09 08:08:14.709-08,NaN,NaN,f,NaN,NaN
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN
2017,2535,Devon Snyder,2535,479,University District,Occlusion,47.659653,-122.319908,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:36.919-08,NaN,NaN,f,NaN,NaN
2018,2536,Devon Snyder,2536,479,University District,SurfaceProblem,47.659668,-122.319916,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:40.146-08,NaN,2.0,f,cracks,NaN


In [4]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104)
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103)
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106)
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN,POINT (-122.32072 47.66111)
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2533,Devon Snyder,2533,479,University District,Occlusion,47.659710,-122.319855,bWNcSsunJ_MyQcFwru1edg,2022-11-09 08:08:14.709-08,NaN,NaN,f,NaN,NaN,POINT (-122.31985 47.65971)
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN,POINT (-122.31992 47.65972)
2017,2535,Devon Snyder,2535,479,University District,Occlusion,47.659653,-122.319908,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:36.919-08,NaN,NaN,f,NaN,NaN,POINT (-122.31991 47.65965)
2018,2536,Devon Snyder,2536,479,University District,SurfaceProblem,47.659668,-122.319916,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:40.146-08,NaN,2.0,f,cracks,NaN,POINT (-122.31992 47.65967)


In [5]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [6]:
#export to json
# chicago.to_file("data/chicago-validation-labels-round2-230608.geojson", driver='GeoJSON')
# seattle.to_file("data/seattle-validation-labels-round2-230608.geojson", driver='GeoJSON')

In [8]:
# read in datasets/streets_all_230515.json
streets_validated = gpd.read_file("data/round-1/mikey-devon-streets-round-1.csv")
streets_validated

,route_id,region_id,street_edge_id,way_type,geom,geometry
0,,9,13,residential,0102000020E6100000030000001192600553995EC08DAF...,None
1,,9,35,residential,0102000020E610000002000000D08BC9D152995EC06957...,None
2,,9,37,residential,0102000020E61000000200000017AE580D53995EC0F805...,None
3,,9,44,tertiary,0102000020E610000003000000836E8A227F995EC0A567...,None
4,,1,72,residential,0102000020E6100000050000002F432D173B975EC059AC...,None
...,...,...,...,...,...,...
76,,3,751,residential,0102000020E6100000040000001E66D421DC905EC04530...,None
77,,3,810,residential,0102000020E61000000400000063CB97BC26915EC0C5D6...,None
78,,3,811,residential,0102000020E610000002000000D9BDCBFBEE905EC05BBC...,None
79,,7,842,residential,0102000020E610000003000000D5100B6B19915EC01164...,None


In [9]:
from shapely import wkb

In [10]:
# apply wkb.loads(geom, hex=True) to the geom column
streets_validated['geom'] = streets_validated['geom'].apply(lambda x: wkb.loads(x, hex=True))
streets_validated

,route_id,region_id,street_edge_id,way_type,geom,geometry
0,,9,13,residential,"LINESTRING (-122.3956922 47.6773064, -122.3963...",None
1,,9,35,residential,"LINESTRING (-122.3956799 47.6765936, -122.3956...",None
2,,9,37,residential,"LINESTRING (-122.3956941 47.6758792, -122.3956...",None
3,,9,44,tertiary,"LINESTRING (-122.3983847 47.677293, -122.39839...",None
4,,1,72,residential,"LINESTRING (-122.3629816 47.5501056, -122.3628...",None
...,...,...,...,...,...,...
76,,3,751,residential,"LINESTRING (-122.2634358 47.6684541, -122.2634...",None
77,,3,810,residential,"LINESTRING (-122.2679893 47.6722212, -122.2677...",None
78,,3,811,residential,"LINESTRING (-122.2645864 47.6721052, -122.2635...",None
79,,7,842,residential,"LINESTRING (-122.2671764 47.5415918, -122.2657...",None


In [11]:
# create streets geo dataframe
streets_validated = gpd.GeoDataFrame(streets_validated, geometry='geom')
streets_validated.crs = {'init': 'epsg:4326'}
streets_validated.explore()


In [12]:
#seperate seattle and chicago based on geometry, chicago >-105 and < -80, seattle < -105
streets_chicago = streets_validated.loc[streets_validated['geom'].bounds['minx'] > -105]
streets_chicago = streets_chicago.loc[streets_chicago['geom'].bounds['minx'] < -80]
streets_seattle = streets_validated.loc[streets_validated['geom'].bounds['minx'] < -105]

In [42]:
chicago

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry
76,94,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:14.398-07,t,4.0,f,"not enough landing space,narrow,not level with...",NaN,POINT (-87.72816 42.02261)
77,96,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:58.603-07,t,3.0,f,"not enough landing space,surface problem",NaN,POINT (-87.72815 42.02272)
78,97,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:57:43.381-07,t,3.0,f,"not enough landing space,points into traffic",NaN,POINT (-87.72843 42.02272)
79,98,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:58:11.132-07,t,3.0,f,not enough landing space,NaN,POINT (-87.72844 42.02262)
80,99,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,POINT (-87.72815 42.02300)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900,2386,mikey,2386,447,Skokie,SurfaceProblem,42.024151,-87.728142,AAY7v8bGlzT1XFHSxQ_jKQ,2022-07-05 12:41:31.117-07,NaN,1.0,f,grass,NaN,POINT (-87.72814 42.02415)
1901,2387,mikey,2387,447,Skokie,CurbRamp,42.023594,-87.728142,tagGeC6m-TMnzcfHLPX6Hg,2022-07-05 12:41:55.266-07,NaN,1.0,f,NaN,NaN,POINT (-87.72814 42.02359)
1902,2388,mikey,2388,447,Skokie,CurbRamp,42.023453,-87.728127,TdCToHEwW86oaUfwfNNDaw,2022-07-05 12:42:00.438-07,NaN,1.0,f,NaN,NaN,POINT (-87.72813 42.02345)
1903,2389,mikey,2389,447,Skokie,CurbRamp,42.022594,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-07-05 12:42:51.976-07,NaN,1.0,f,NaN,NaN,POINT (-87.72815 42.02259)


In [13]:
base = streets_chicago.explore( tiles='cartodbpositron', color='red')

In [14]:
base_2 = streets_seattle.explore( tiles='cartodbpositron', color='red')

In [15]:
test = chicago.explore(m= base, color='blue', tiles='cartodbpositron')

In [16]:
test_2 = seattle.explore(m= base_2, color='blue', tiles='cartodbpositron')

In [17]:
test_2

In [19]:
test.save('chicago_round_1_230609_evening.html')

In [18]:
test_2.save('seattle_round_1_230609_evening.html')

In [12]:
#save to geojson
streets_chicago.to_file("data/chicago-streets-230608.geojson", driver='GeoJSON')
streets_seattle.to_file("data/seattle-streets-230608.geojson", driver='GeoJSON')